# CATECHOL BENCHMARK: SYMMETRIC KINETIC ENSEMBLE [Score: 0.09831]

This notebook improves upon standard baselines by incorporating two Physics-Informed priors:

1. **ARRHENIUS KINETICS:** We explicitly feed 1/Temperature and ln(Time) to the model.
2. **CHEMICAL SYMMETRY:** A mixture of "Solvent A + Solvent B" is physically identical to "Solvent B + Solvent A". Standard models treat them differently.


**THE TRICK (Test Time Augmentation):**
During inference for mixed solvents, we predict the target twice:\
   Prediction 1: Input as (A, B)\
   Prediction 2: Input as (B, A) (Flipped)\
   Final = (Pred1 + Pred2) / 2

This mathematically guarantees that our model respects the physical symmetry of the mixture,
reducing variance and improving the score to ***0.09831***.

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import sys
import tqdm

# --- CONFIGURATION ---
sys.path.append('/kaggle/input/catechol-benchmark-hackathon/')
torch.set_default_dtype(torch.double)
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

# --- UTILS ---
def load_data(name="full"):
    df = pd.read_csv(f'/kaggle/input/catechol-benchmark-hackathon/catechol_{"full_data_yields" if name=="full" else "single_solvent_yields"}.csv')
    if name == "full":
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features():
    # Spange descriptors provide the best base for linear mixing
    return pd.read_csv('/kaggle/input/catechol-benchmark-hackathon/spange_descriptors_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

SPANGE_DF = load_features()

# --- 1. KINETIC FEATURIZER ---
class SmilesFeaturizer(ABC):
    def featurize(self, X, flip=False): raise NotImplementedError

class KineticMixingFeaturizer(SmilesFeaturizer):
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        self.feats_dim = self.featurizer.shape[1] + 2 + 3

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # --- KINETIC ENGINEERING ---
        temp_c = X_vals[:, 1:2] 
        time_m = X_vals[:, 0:1]
        
        temp_k = temp_c + 273.15 
        inv_temp = 1000.0 / temp_k 
        log_time = np.log(time_m + 1e-6) 
        interaction = inv_temp * log_time 
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # --- SYMMETRIC CHEMICAL FEATURES ---
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # SYMMETRY FLIP: Swap A and B, use (1-p) as concentration
                # Chemically: A(p) + B(1-p) is same as B(1-p) + A(p)
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

# --- 2. ROBUST MLP ---
class MLPInternal(nn.Module):
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid() 
        )

    def forward(self, x):
        return self.net(x)

# --- 3. SYMMETRIC BAGGED MODEL ---
class SymmetricBaggedModel(nn.Module):
    def __init__(self, data='single'):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        
        # 7 Models for Bagging
        self.n_models = 7
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train):
        # 1. Standard Data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # 2. Augmented Data (Train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Training {self.n_models} models on {device}...")
        
        for i in range(self.n_models):
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss() # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(300):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        # --- TEST TIME AUGMENTATION (TTA) ---
        if self.data_type == 'full':
            # Path A: Standard
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            # Path B: Flipped (Symmetry)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    # Average the model's prediction on A and B views
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5 
            
            avg_pred = pred_sum / self.n_models
            
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

# --- 4. EXECUTION ---
# Task 0: Single Solvent
print("\n--- TASK 0: Single Solvent ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    model = SymmetricBaggedModel(data='single')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

# Task 1: Full Data
print("\n--- TASK 1: Full Data (Mixtures) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    model = SymmetricBaggedModel(data='full')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

# Save
submission = pd.DataFrame(all_predictions)
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)
print("Submission Generated.")